In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # noqa
import torch.optim as optim

from torchvision import datasets, transforms

In [3]:
torch.manual_seed(4242)

In [6]:
train_loader = torch.utils.data.DataLoader(  # noqa
    datasets.MNIST("../data/p1ch2/mnist", train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True,
)

In [8]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
model = Net()

In [10]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [11]:
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

    print("Current loss", float(loss))

Current loss 0.5039969682693481
Current loss 0.1340223103761673
Current loss 0.42982757091522217
Current loss 0.4821724593639374
Current loss 0.20132414996623993
Current loss 0.30001986026763916
Current loss 0.23043106496334076
Current loss 0.25843164324760437
Current loss 0.027089498937129974
Current loss 0.0640357956290245


In [12]:
torch.save(model.state_dict(), "../data/p1ch2/mnist/mnist.pth")

In [13]:
pretrained_model = Net()

In [1]:
pretrained_model.load_state_dict(torch.load("../data/p1ch2/mnist/mnist.pth"))

NameError: name 'pretrained_model' is not defined